# ZenML Quickstart

Welcome to ZenML! This interactive notebook guides you through the core concepts of building and deploying ML pipelines.

We'll run a simple pipeline locally, create snapshots, deploy it as an HTTP service, and explore cloud infrastructure options.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/quickstart.ipynb)

## Step 0: Install ZenML and Setup

First, let's install ZenML and clone the quickstart example if you're on Colab.

In [ ]:
!pip install -q "zenml[server]"

from zenml.environment import Environment

if Environment.in_google_colab():
    # Install Cloudflare Tunnel binary for public endpoint
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

    # Clone the quickstart example
    !git clone -q -b main https://github.com/zenml-io/zenml
    !cp -r zenml/examples/quickstart/* . && rm -rf zenml

print("✅ ZenML installed successfully!")

## Step 1: Create Your First Pipeline

Let's build a simple pipeline! A ZenML pipeline is just a function decorated with `@pipeline` that orchestrates steps.

In [ ]:
from typing import Annotated

from zenml import pipeline, step


@step
def simple_step(name: str = "World") -> Annotated[str, "greeting"]:
    """A simple step that returns a greeting."""
    message = f"Hello, {name}! 👋 Welcome to ZenML!"
    return message


@pipeline
def simple_pipeline(name: str = "World") -> Annotated[str, "greeting"]:
    """A simple pipeline with one step."""
    return simple_step(name=name)


print("✅ Pipeline defined!")

## Step 2: Run Your Pipeline Locally

Now let's execute the pipeline. ZenML automatically tracks the execution, artifacts, and metadata.

In [ ]:
print("🚀 Running pipeline...")
result = simple_pipeline(name="ZenML User")
print(f"\n✅ Pipeline completed!")
print(f"Run ID: {result.id}")
print(f"Status: {result.status}")

## Step 3: Start ZenML Server and View Dashboard

Your pipeline run is now tracked in ZenML! Let's start the ZenML server to see the dashboard with your execution history, artifacts, and metadata.

In [ ]:
from zenml.client import Client
from zenml.environment import Environment
from zenml.zen_stores.rest_zen_store import RestZenStore

client = Client()

if not isinstance(client.zen_store, RestZenStore):
    # Only spin up a local Dashboard in case you aren't already connected to a remote server
    if Environment.in_google_colab():
        # run ZenML through a cloudflare tunnel to get a public endpoint
        !zenml login --local --port 8237 & cloudflared tunnel --url http://localhost:8237
    else:
        !zenml login --local

print("✅ ZenML server started!")
print("Dashboard URL: http://localhost:8237")

## Step 4: Why Import from Modules for Snapshots and Deployments?

For snapshots and deployments to work properly, ZenML needs to serialize and load your pipeline code from actual Python modules, not from notebook cells.

**In this step**, we'll import the pipeline from the actual `pipelines/simple_pipeline.py` file to prepare for snapshots and deployments. This is a best practice for production workflows.

In [ ]:
from pipelines.simple_pipeline import simple_pipeline as simple_pipeline_module

# Use the module-based pipeline for snapshots and deployments
simple_pipeline = simple_pipeline_module

print("✅ Imported pipeline from module!")

## Step 5: Create a Pipeline Snapshot

Snapshots make your pipelines reproducible by freezing the code, configuration, and container image. Create a snapshot directly with the Python SDK:

In [ ]:
# Create a snapshot of the pipeline
print("📸 Creating a pipeline snapshot...")
snapshot = simple_pipeline.create_snapshot(name="simple_snapshot")
print(f"✅ Snapshot created!")
print(f"Snapshot ID: {snapshot.id}")
print(f"Snapshot Name: {snapshot.name}")
print(f"\nSnapshots freeze your pipeline state for:")
print(f"  • Reproducibility across runs")
print(f"  • Sharing with team members")
print(f"  • Deploying as production services")

## Step 6: Deploy Your Pipeline as an HTTP Service

Transform your pipeline into a real-time HTTP service for live inference. Use the Python SDK to deploy directly:

In [ ]:
# Deploy the pipeline as an HTTP service
print("🚀 Deploying pipeline as HTTP service...")
deployment = simple_pipeline.deploy(deployment_name="simple_deployment")
print(f"✅ Deployment created!")
print(f"Deployment URL: {deployment.url}")
print(f"Status: {deployment.status}")
print(f"\n📋 Your pipeline is now a REST API!")
print(f"\nInvoke it with curl:")
print(f"  curl -X POST {deployment.url}/invoke \\")
print(f"    -H 'Content-Type: application/json' \\")
print(f'    -d \'{{"parameters": {{"name": "Alice"}}}}\'')

## Step 7: Invoke Your Deployed Service

Now let's call our deployed service to process a request:

In [ ]:
from zenml.deployers.utils import invoke_deployment

print("🌐 Invoking deployed service...")
response = invoke_deployment(
    deployment_name_or_id="simple_deployment", name="Friend"
)
print(f"✅ Service response received!")
print(f"\nResponse:")
print(f"  Status: {response.get('success')}")
print(f"  Output: {response.get('outputs', {}).get('greeting', 'N/A')}")
print(f"  Execution Time: {response.get('execution_time')}s")

# ZenML Quickstart - Summary

You've successfully learned the core ZenML concepts by running a complete workflow locally!

## What You've Accomplished

✅ **Pipelines & Steps** - Built composable workflows with automatic tracking
✅ **Local Execution** - Ran pipelines with automatic artifact management
✅ **Snapshots** - Froze pipeline state for reproducibility
✅ **Deployments** - Transformed your pipeline into an HTTP service
✅ **Dashboard** - Inspected runs and metadata in the ZenML UI

## Running on Cloud Infrastructure

To run this same pipeline on cloud orchestrators (AWS, GCP, Azure):
1. Deploy a ZenML server (ZenML Pro or self-hosted)
2. Create a remote stack with cloud components
3. Run the same code - ZenML handles containerization and infrastructure

Your code doesn't change - just switch your stack!

## Next Steps

Explore other examples to see ZenML in action:
- **[Deploying ML Models](https://github.com/zenml-io/zenml/tree/main/examples/deploying_ml_model)** - Production ML model serving
- **[Deploying Agents](https://github.com/zenml-io/zenml/tree/main/examples/deploying_agent)** - LLM-powered agents
- **[Agent Outer Loop](https://github.com/zenml-io/zenml/tree/main/examples/agent_outer_loop)** - Hybrid ML + agent systems
- **[Full Documentation](https://docs.zenml.io/)** - Deep dives into all ZenML features